## Access this Notebook
You can launch this notebook in the US GHG Center JupyterHub by clicking the link below. If you are a new user, you should first sign up for the hub by filling out this [**request form**](https://docs.google.com/forms/d/e/1FAIpQLSdai8otCdrVQzJgev8mjDhzKyCg7jcrB3UeTXNHoCiaMKrkaQ/viewform) and providing the required information. 


Access the [**SEDAC Gridded World Population Density**](https://us-ghg-center.github.io/ghgc-docs/user_data_notebooks/sedac-popdensity-yeargrid5yr-v4.html) notebook in the US GHG Center JupyterHub. 

## Table of Contents
- [Data Summary and Application](#data-summary-and-application)
- [Approach](#approach)
- [About the Data](#about-the-data)
- [Install the Required Libraries](#install-the-required-libraries)
- [Query the STAC API](#query-the-stac-api)
- [Visual Comparison Across Time Periods](#visual-comparison-across-time-periods)
- [Map Out Selected Tiles](#map-out-selected-tiles)
- [Calculate Zonal Statistics](#calculate-zonal-statistics)
- [Time-Series Analysis](#time-series-analysis)
- [Summary](#summary)

## Data Summary and Application
- **Spatial coverage**: Global
- **Spatial resolution**: 30 arc-seconds (~1 km at equator)
- **Temporal extent**: 2000 - 2020
- **Temporal resolution**: Annual, every 5 years
- **Unit**: Number of persons per square kilometer
- **Utility**: Climate Research

For more, visit the [SEDAC Gridded World Population Density](https://earth.gov/ghgcenter/data-catalog/sedac-popdensity-yeargrid5yr-v4.11) data overview page.  

## Approach

   1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. Collection processed in this notebook is SEDAC gridded population density.
   2. Pass the STAC item into raster API `/collections/{collection_id}/items/{item_id}/tilejson.json` endpoint
   3. We'll visualize two tiles (side-by-side) allowing for comparison of each of the time points using `folium.plugins.DualMap`
   4. After the visualization, we'll perform zonal statistics for a given polygon.
   

## About the Data

The SEDAC Gridded Population of the World: Population Density, v4.11 dataset provides annual estimates of population density for the years 2000, 2005, 2010, 2015, and 2020 on a 30 arc-second (~1 km) grid. These data can be used for assessing disaster impacts, risk mapping, and any other applications that include a human dimension. This population density dataset is provided by NASA’s Socioeconomic Data and Applications Center (SEDAC) hosted by the Center for International Earth Science Information Network (CIESIN) at Columbia University. The population estimates are provided as a continuous raster for the entire globe.

For more information regarding this dataset, please visit the [SEDAC Gridded World Population Density](https://earth.gov/ghgcenter/data-catalog/sedac-popdensity-yeargrid5yr-v4.11) data overview page.

# Install the Required Libraries
Required libraries are pre-installed on the GHG Center Hub. If you need to run this notebook elsewhere, please install them with this line in a code cell:

%pip install requests folium rasterstats pystac_client pandas matplotlib --quiet

In [1]:
# Import the following libraries
import requests
import folium
import folium.plugins
from folium import Map, TileLayer
from pystac_client import Client
import branca
import pandas as pd
import matplotlib.pyplot as plt
from stats_module import generate_stats, clean_stats, display_stats

## Query the STAC API
First, we are going to import the required libraries. Once imported, they allow better executing a query in the GHG Center Spatio Temporal Asset Catalog (STAC) Application Programming Interface (API) where the granules for this collection are stored.

In [2]:
# Provide the STAC and RASTER API endpoints
# The endpoint is referring to a location within the API that executes a request on a data collection nesting on the server.

# The STAC API is a catalog of all the existing data collections that are stored in the GHG Center.
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"

# The RASTER API is used to fetch collections for visualization
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
# Name of the collection for SEDAC population density dataset 
collection_name = "sedac-popdensity-yeargrid5yr-v4.11"

In [3]:
# Using PySTAC client
# Fetch the collection from the STAC API using the appropriate endpoint
# The 'pystac' library allows a HTTP request possible
catalog = Client.open(STAC_API_URL)
collection = catalog.get_collection(collection_name)

# Print the properties of the collection to the console
collection

<CollectionClient id=sedac-popdensity-yeargrid5yr-v4.11>

Examining the contents of our `collection` under `summaries` we see that the data is available from January 2000 to December 2020. By looking at the `dashboard:time density` we observe that the data is available for the years 2000, 2005, 2010, 2015, 2020. 

In [4]:
items = list(collection.get_items())  # Convert the iterator to a list
print(f"Found {len(items)} items")

Found 5 items


In [5]:
# Examine the first item in the collection
# Keep in mind that a list starts from 0, 1, 2... therefore items[0] is referring to the first item in the list/collection
items[0]

<Item id=sedac-popdensity-yeargrid5yr-v4.11-gpw_v4_population_density_rev11_2020_30_sec_2020>

## Visual Comparison Across Time Periods

You will explore changes in population density in urban regions. In this notebook, we'll explore the changes in population density over time.You will visualize the outputs on a map using `folium`. 

In [6]:
# Now we create a dictionary where the start datetime values for each granule is queried more explicitly by year and month (e.g., 2020-02)
# To access the year value from each item more easily, this will let us query more explicitly by year and month (e.g., 2020-02)
items = {item.properties["start_datetime"][:7]: item for item in collection.get_items()}

# Next, we need to specify the asset name for this collection
# The asset name is referring to the raster band containing the pixel values for the parameter of interest
# For the case of the SEDAC Gridded World Population Density collection, the parameter of interest is “population-density”
asset_name = "population-density"

Now, we will pass the item id, collection name, asset name, and the `rescaling factor` to the `Raster API` endpoint. We will do this twice, once for January 2000 and again for January 2020, so that we can visualize each event independently. 

In [7]:
# Choose a color map for displaying the first observation (event)
# Please refer to matplotlib library if you'd prefer choosing a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html
color_map = "rainbow" 

# Make a GET request to retrieve information for the date mentioned below
first_date = items["2020-01"]

# Extract collection name and item ID
collection_id = first_date.collection_id
item_id = first_date.id

object = first_date.assets[asset_name]
raster_bands = object.extra_fields.get("raster:bands", [{}])
rescale_values = {
    "max": raster_bands[0].get("histogram", {}).get("max"),
    "min": raster_bands[0].get("histogram", {}).get("min"),
}

print(rescale_values)

{'max': 30795.859375, 'min': -1505.7174072265625}


In [8]:
# Make a GET request to retrieve information for the date mentioned below
month1_tile = requests.get(
    f"{RASTER_API_URL}/collections/{collection_id}/items/{item_id}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}"
).json()

# Print the properties of the retrieved granule to the console
month1_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://earth.gov/ghgcenter/api/raster/collections/sedac-popdensity-yeargrid5yr-v4.11/items/sedac-popdensity-yeargrid5yr-v4.11-gpw_v4_population_density_rev11_2020_30_sec_2020/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=population-density&color_formula=gamma+r+1.05&colormap_name=rainbow&rescale=-1505.7174072265625%2C30795.859375'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 179.99999999999983, 89.99999999999991],
 'center': [-8.526512829121202e-14, -4.263256414560601e-14, 0]}

In [9]:
# Make a GET request to retrieve information for date mentioned below
second_date = items["2000-01"]

# Extract collection name and item ID
collection_id = second_date.collection_id
item_id = second_date.id

object = second_date.assets[asset_name]
raster_bands = object.extra_fields.get("raster:bands", [{}])
rescale_values = {
    "max": raster_bands[0].get("histogram", {}).get("max"),
    "min": raster_bands[0].get("histogram", {}).get("min"),
}

print(rescale_values)

month2_tile = requests.get(
    f"{RASTER_API_URL}/collections/{collection_id}/items/{item_id}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}"
).json()

# Print the properties of the retrieved granule to the console
month2_tile

{'max': 20757.5234375, 'min': -800.1041259765625}


{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://earth.gov/ghgcenter/api/raster/collections/sedac-popdensity-yeargrid5yr-v4.11/items/sedac-popdensity-yeargrid5yr-v4.11-gpw_v4_population_density_rev11_2000_30_sec_2000/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=population-density&color_formula=gamma+r+1.05&colormap_name=rainbow&rescale=-800.1041259765625%2C20757.5234375'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 179.99999999999983, 89.99999999999991],
 'center': [-8.526512829121202e-14, -4.263256414560601e-14, 0]}

## Map Out Selected Tiles

In [10]:
# Set initial zoom and center of map for population density Layer
# 'folium.plugins' allows mapping side-by-side
map_ = folium.plugins.DualMap(location=(34, -118), zoom_start=9) #Los Angeles

# Define the first map layer (January 2020)
map_layer_2020 = TileLayer(
    tiles=month1_tile["tiles"][0], # Path to retrieve the tile
    attr="GHG", # Set the attribution
    opacity=1, # Adjust the transparency of the layer
)

# Add the first layer to the Dual Map
map_layer_2020.add_to(map_.m1)

# Define the second map layer (January 2000)
map_layer_2000 = TileLayer(
    tiles=month2_tile["tiles"][0], # Path to retrieve the tile
    attr="GHG", # Set the attribution
    opacity=1, # Adjust the transparency of the layer
)

# Add the second layer to the Dual Map
map_layer_2000.add_to(map_.m2)

# Visualize the Dual Map
map_

# Calculating Zonal Statistics
To perform zonal statistics, first we need to create a polygon. In this use case we are creating a polygon in Texas (USA).

In [11]:
# San Francisco, CA USA
# Create a polygon for the area of interest (aoi)
sanfrancisco_aoi = {
    "type": "Feature", # Create a feature object
    "properties": {},
    "geometry": { # Set the bounding coordinates for the polygon
        "coordinates": [
            [
                [-122.52709933248232, 37.816224913955665],
                [-122.52709933248232,37.69714432421999],
                [-122.3493837208909,37.69714432421999],
                [-122.3493837208909,37.816224913955665],
                [-122.52709933248232,37.816224913955665]            
            ]
        ],
        "type": "Polygon",
    },
}

In [12]:
# Check total number of items available within the collection
items = requests.get(
    f"{STAC_API_URL}/collections/{collection_name}/items?limit=300"
).json()["features"]

# Print the total number of items (granules) found
print(f"Found {len(items)} items")

Found 5 items


In [13]:
# Examine the first item in the collection
items[0]

{'id': 'sedac-popdensity-yeargrid5yr-v4.11-gpw_v4_population_density_rev11_2020_30_sec_2020',
 'bbox': [-180.0, -90.0, 179.99999999999983, 89.99999999999991],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/sedac-popdensity-yeargrid5yr-v4.11'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/sedac-popdensity-yeargrid5yr-v4.11'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/sedac-popdensity-yeargrid5yr-v4.11/items/sedac-popdensity-yeargrid5yr-v4.11-gpw_v4_population_density_rev11_2020_30_sec_2020'},
  {'title': 'Map of Item',
   'href': 'https://earth.gov/ghgcenter/api/raster/collections/sedac-popdensity-yeargrid5yr-v4.11/items/sedac-popdensity-yeargrid5yr-v4.1

Now that we created the polygon for the area of interest, we need to develop a function that runs through the data collection and generates the statistics for a specific item (granule) within the boundaries of the AOI polygon.

With the function above we can generate the statistics for the AOI.

In [15]:
%%time
# %%time = Wall time (execution time) for running the code below

# Generate statistics using the created function "generate_stats" within the bounding box defined by the polygon
stats = [generate_stats(item, sanfrancisco_aoi,asset_name) for item in items]

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-122.52709933248232, 37.816224913955665], [-122.52709933248232, 37.69714432421999], [-122.3493837208909, 37.69714432421999], [-122.3493837208909, 37.816224913955665], [-122.52709933248232, 37.816224913955665]]]}, 'properties': {'statistics': {'b1': {'min': 0.0, 'max': 38470.6796875, 'mean': 5629.80859375, 'count': 233.39999389648438, 'sum': 1313997.25, 'std': 5291.390932448669, 'median': 5040.76953125, 'majority': 0.0, 'minority': 0.009292405098676682, 'unique': 229.0, 'histogram': [[109, 60, 53, 13, 4, 3, 1, 0, 0, 1], [0.0, 3847.06787109375, 7694.1357421875, 11541.203125, 15388.271484375, 19235.33984375, 23082.40625, 26929.474609375, 30776.54296875, 34623.609375, 38470.6796875]], 'valid_percent': 70.72, 'masked_pixels': 101.0, 'valid_pixels': 244.0, 'percentile_2': 0.0, 'percentile_98': 20110.85546875}}}}
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-122.52709933248232, 37.816224913955665], 

In [16]:
# Print the stats for the first item in the collection
stats[0]

{'statistics': {'b1': {'min': 0.0,
   'max': 38470.6796875,
   'mean': 5629.80859375,
   'count': 233.39999389648438,
   'sum': 1313997.25,
   'std': 5291.390932448669,
   'median': 5040.76953125,
   'majority': 0.0,
   'minority': 0.009292405098676682,
   'unique': 229.0,
   'histogram': [[109, 60, 53, 13, 4, 3, 1, 0, 0, 1],
    [0.0,
     3847.06787109375,
     7694.1357421875,
     11541.203125,
     15388.271484375,
     19235.33984375,
     23082.40625,
     26929.474609375,
     30776.54296875,
     34623.609375,
     38470.6796875]],
   'valid_percent': 70.72,
   'masked_pixels': 101.0,
   'valid_pixels': 244.0,
   'percentile_2': 0.0,
   'percentile_98': 20110.85546875}},
 'datetime': '2020-01-01T00:00:00+00:00'}

Create a function that goes through every single item in the collection and populates their properties - including the minimum, maximum, and sum of their values - in a table.

In [17]:
# Apply the generated function on the stats data
df = clean_stats(stats)

# Display the stats for the first 5 granules in the collection in the table
# Change the value in the parenthesis to show more or a smaller number of rows in the table
df.head(5)

,datetime,min,max,mean,count,sum,std,median,majority,minority,unique,histogram,valid_percent,masked_pixels,valid_pixels,percentile_2,percentile_98,date
0,2020-01-01T00:00:00+00:00,0.0,38470.679688,5629.808594,233.399994,1313997.250,5291.390932,5040.769531,0.0,0.009292,229.0,"[[109, 60, 53, 13, 4, 3, 1, 0, 0, 1], [0.0, 38...",70.72,101.0,244.0,0.0,20110.855469,2020-01-01 00:00:00+00:00
1,2015-01-01T00:00:00+00:00,0.0,37784.449219,5532.063477,233.399994,1291183.625,5197.331238,4950.853516,0.0,0.009127,229.0,"[[109, 60, 53, 13, 4, 3, 1, 0, 0, 1], [0.0, 37...",70.72,101.0,244.0,0.0,19752.123047,2015-01-01 00:00:00+00:00
2,2010-01-01T00:00:00+00:00,0.0,37110.457031,5436.040527,233.399994,1268771.875,5104.976396,4862.541504,0.0,0.008964,229.0,"[[109, 60, 53, 13, 4, 3, 1, 0, 0, 1], [0.0, 37...",70.72,101.0,244.0,0.0,19399.789062,2010-01-01 00:00:00+00:00
3,2005-01-01T00:00:00+00:00,0.0,36448.488281,5341.709473,233.399994,1246755.000,5014.296361,4775.804199,0.0,0.008804,229.0,"[[109, 60, 53, 13, 4, 3, 1, 0, 0, 1], [0.0, 36...",70.72,101.0,244.0,0.0,19053.738281,2005-01-01 00:00:00+00:00
4,2000-01-01T00:00:00+00:00,0.0,35798.328125,5249.041016,233.399994,1225126.125,4925.261211,4690.614258,0.0,0.008647,230.0,"[[109, 60, 53, 13, 4, 3, 1, 0, 0, 1], [0.0, 35...",70.72,101.0,244.0,0.0,18713.861328,2000-01-01 00:00:00+00:00


## Time-Series Analysis
We can now explore the SEDAC population density dataset time series available for the Texas, Dallas area of USA. We can plot the dataset using the code below:

In [18]:
# Figure size: 20 representing the width, 10 representing the height
fig = plt.figure(figsize=(20, 10))

plt.plot(
    df["date"], # X-axis: sorted datetime
    df["max"], # Y-axis: maximum pop density
    color="red", # Line color
    linestyle="-", # Line style
    linewidth=0.5, # Line width
    label="Population density over the years", # Legend label
)

# Display legend
plt.legend()

# Insert label for the X-axis
plt.xlabel("Years")

# Insert label for the Y-axis
plt.ylabel("Population density")

# Insert title for the plot
plt.title("Population density over San Francisco, CA USA (2000-2020)")

###
# Add data citation
plt.text(
    df["date"].iloc[0],           # X-coordinate of the text
    df["max"].min(),              # Y-coordinate of the text




    # Text to be displayed
    "Source: NASA SEDAC Gridded World Population Density",                  
    fontsize=12,                             # Font size
    horizontalalignment="right",             # Horizontal alignment
    verticalalignment="bottom",              # Vertical alignment
    color="blue",                            # Text color
)


# Plot the time series
plt.show()

## Summary

In this notebook we have successfully explored, analyzed and visualized the STAC collection for the SEDAC Gridded World Population Density dataset.

1.  Install and import the necessary libraries
2.  Fetch the collection from STAC collections using the appropriate endpoints
3.  Count the number of existing granules within the collection
4.  Map and compare population density for two distinctive months/years
5.  Generate zonal statistics for the area of interest (AOI)
6.  Visualizing the Data as a Time Series

If you have any questions regarding this user notebook, please contact us using the [feedback form](https://docs.google.com/forms/d/e/1FAIpQLSeVWCrnca08Gt_qoWYjTo6gnj1BEGL4NCUC9VEiQnXA02gzVQ/viewform).